# Exploring the NASA Exoplanet Archive planetary properties data

In [3]:
import numpy as np
import pandas as pd
import os
import glob
from astropy.io import fits
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.time import Time
import lightkurve as lk
from astropy.table import Table

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_context('paper')

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [4]:
table_1 = Table.read('../data/Exoplanet_Finder/PS_2021.04.05_08.52.09.votable',format='votable')

In [5]:
df_orig = table_1.to_pandas()

In [6]:
df_orig.tail()

pl_name hostname default_flag sy_snum sy_pnum  discoverymethod  \
29382  ups And d  ups And            0       2       3  Radial Velocity   
29383  ups And d  ups And            1       2       3  Radial Velocity   
29384  ups And d  ups And            0       2       3  Radial Velocity   
29385   xi Aql b   xi Aql            1       1       1  Radial Velocity   
29386   xi Aql b   xi Aql            0       1       1  Radial Velocity   

      disc_year                      disc_facility              soltype  \
29382      1999             Multiple Observatories  Published Confirmed   
29383      1999             Multiple Observatories  Published Confirmed   
29384      1999             Multiple Observatories  Published Confirmed   
29385      2007  Okayama Astrophysical Observatory  Published Confirmed   
29386      2007  Okayama Astrophysical Observatory  Published Confirmed   

      pl_controv_flag  ... sy_vmagerr2  sy_kmag  sy_kmagerr1  sy_kmagerr2  \
29382               0  ...      -0.023    2.859        0.274       -0.274   
29383               0  ...      -0.023    2.859        0.274       -0.274   
29384               0  ...      -0.023    2.859        0.274       -0.274   
29385               0  ...      -0.023    2.171        0.220       -0.220   
29386               0  ...      -0.023    2.171        0.220       -0.220   

      sy_gaiamag  sy_gaiamagerr1  sy_gaiamagerr2   rowupdate pl_pubdate  \
29382    3.98687        0.008937       -0.008937  2014-08-21    2004-01   
29383    3.98687        0.008937       -0.008937  2019-01-28    2011-01   
29384    3.98687        0.008937       -0.008937  2014-09-18    2007-01   
29385    4.42501        0.003837       -0.003837  2014-05-14    2008-06   
29386    4.42501        0.003837       -0.003837  2014-07-23    2011-08   

       releasedate  
29382   2014-08-21  
29383   2019-01-31  
29384   2014-09-18  
29385   2014-05-14  
29386   2014-07-23  

[5 rows x 92 columns]

In [7]:
list(df_orig.columns)

['pl_name',
 'hostname',
 'default_flag',
 'sy_snum',
 'sy_pnum',
 'discoverymethod',
 'disc_year',
 'disc_facility',
 'soltype',
 'pl_controv_flag',
 'pl_refname',
 'pl_orbper',
 'pl_orbpererr1',
 'pl_orbpererr2',
 'pl_orbperlim',
 'pl_orbsmax',
 'pl_orbsmaxerr1',
 'pl_orbsmaxerr2',
 'pl_orbsmaxlim',
 'pl_rade',
 'pl_radeerr1',
 'pl_radeerr2',
 'pl_radelim',
 'pl_radj',
 'pl_radjerr1',
 'pl_radjerr2',
 'pl_radjlim',
 'pl_bmasse',
 'pl_bmasseerr1',
 'pl_bmasseerr2',
 'pl_bmasselim',
 'pl_bmassj',
 'pl_bmassjerr1',
 'pl_bmassjerr2',
 'pl_bmassjlim',
 'pl_bmassprov',
 'pl_orbeccen',
 'pl_orbeccenerr1',
 'pl_orbeccenerr2',
 'pl_orbeccenlim',
 'pl_insol',
 'pl_insolerr1',
 'pl_insolerr2',
 'pl_insollim',
 'pl_eqt',
 'pl_eqterr1',
 'pl_eqterr2',
 'pl_eqtlim',
 'ttv_flag',
 'st_refname',
 'st_spectype',
 'st_teff',
 'st_tefferr1',
 'st_tefferr2',
 'st_tefflim',
 'st_rad',
 'st_raderr1',
 'st_raderr2',
 'st_radlim',
 'st_mass',
 'st_masserr1',
 'st_masserr2',
 'st_masslim',
 'st_met',
 'st_me

In [15]:
df=df_orig[['hostname','st_teff','st_rad','st_mass','st_logg','sy_refname']]
df

hostname  st_teff  st_rad  st_mass  st_logg  \
0       11 Com      NaN     NaN     2.60      NaN   
1       11 Com   4742.0   19.00     2.70     2.31   
2       11 UMi   4340.0   24.08     1.80     1.60   
3       11 UMi      NaN     NaN     1.70      NaN   
4       11 UMi   4213.0   29.79     2.78     1.93   
...        ...      ...     ...      ...      ...   
29382  ups And      NaN     NaN      NaN      NaN   
29383  ups And      NaN    1.56     1.30      NaN   
29384  ups And      NaN     NaN      NaN      NaN   
29385   xi Aql   4780.0   12.00     2.20     2.66   
29386   xi Aql      NaN     NaN     1.40      NaN   

                                              sy_refname  
0      <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
1      <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
2      <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
3      <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
4      <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
...                                                  ...  
29382  <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
29383  <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
29384  <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
29385  <a refstr=STASSUN_ET_AL__2019 href=https://ui....  
29386  <a refstr=STASSUN_ET_AL__2019 href=https://ui....  

[29387 rows x 6 columns]